# Xữ lý dữ liệu đầu vào


In [1]:
import pandas as pd

In [3]:
file_path = "Visim-400.txt"
df = pd.read_csv(file_path, sep="\t")
# Lấy cột Word1 và Word2, ghép thành một cột theo thứ tự word1(1), word2(1), word1(2), word2(2)...
words = df[["Word1", "Word2"]].values.flatten()
# Xuất ra file txt, mỗi từ 1 dòng
output_path = "data/words_list2.txt"
with open(output_path, "w", encoding="utf-8") as f:
    for w in words:
        f.write(str(w) + "\n")
print("Đã tạo file:", output_path)

Đã tạo file: words_list2.txt


# Crawl dữ liệu từ BabelNet


In [4]:
import requests
from bs4 import BeautifulSoup
import re
import json
import time
import urllib.parse

### 1.Đọc danh sách các từ

In [6]:
with open("data/words_list2.txt", "r", encoding="utf-8") as f:
    words = [line.strip() for line in f if line.strip()]

### 2.Encode từ để đưa vào URL và crawl các dữ liệu Definitions và các R

In [7]:
results = {}

for word in words:
    try:
        # Encode từ để đưa vào URL
        url = f"https://babelnet.org/search?word={urllib.parse.quote(word)}&lang=VI"
        response = requests.get(url)

        if response.status_code != 200:
            print(f"Lỗi khi lấy từ: {word}")
            continue
        soup = BeautifulSoup(response.content, "html.parser")

        #lấy Definitions
        vi_section = soup.find("div", class_="language-row", attrs={"data-language": "VI"})
        definitions = []
        if vi_section:
            defs = vi_section.find_all("div", class_="definition")
            for d in defs:
                for tag in d.find_all(["span", "a"]):
                    tag.decompose()
                text = d.get_text(" ", strip=True)
                if text:
                    definitions.append(text)

        #Lấy các relations
        relation = soup.find("div", class_="relation-by-type")
        relations_data = {}
        if relation:
            rows = relation.find_all("div", class_="relation-row")
            target_relations = {"IS A", "HAS PART", "PART OF", "HAS KIND", "HAS INSTANCE"}
            for r in rows:
                relation_type = r.find("div", class_="type").get_text(strip=True)
                if relation_type not in target_relations:
                    continue
                colums = r.find("div", class_="relation-list")
                links = colums.find_all("a", class_="relation")
                synonyms = []
                for link in links:
                    synonym = link.find("div", class_="synonim-wrapper")
                    if synonym:
                        word_text = synonym.text.strip()
                        if re.match(r'^[A-Za-z\s\-\(\)]+$', word_text):  # English
                            synonyms.append(word_text + " (EN)")
                        elif re.search(r'[àáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễ'
                                       r'ìíịỉĩòóọỏõôồốộổỗơờớợởỡ'
                                       r'ùúụủũưừứựửữỳýỵỷỹđ]', word_text, re.IGNORECASE):
                            synonyms.append(word_text)  # Tiếng Việt

                relations_data[relation_type] = synonyms
        #Lưu Kết quả
        results[word] = {
            "Definitions": definitions,
            "Relations": relations_data
        }

        print(f"Đã xử lý xong: {word}")
        time.sleep(3)  #thời gian nghỉ để load web

    except Exception as e:
        print(f"Lỗi với từ {word}: {e}")

Đã xử lý xong: biến
Đã xử lý xong: ngập
Đã xử lý xong: nhà_thi_đấu
Đã xử lý xong: nhà
Đã xử lý xong: động
Đã xử lý xong: tĩnh
Đã xử lý xong: khuyết
Đã xử lý xong: ưu
Đã xử lý xong: cõi_tục
Đã xử lý xong: cõi_âm
Đã xử lý xong: thủ_pháp
Đã xử lý xong: biện_pháp
Đã xử lý xong: kết_duyên
Đã xử lý xong: thành_hôn
Đã xử lý xong: cấp_tiến
Đã xử lý xong: bảo_thủ
Đã xử lý xong: nước_lớn
Đã xử lý xong: nguy_hiểm
Đã xử lý xong: hoa
Đã xử lý xong: nhị
Đã xử lý xong: bất_lợi
Đã xử lý xong: thuận_lợi
Đã xử lý xong: phân_ly
Đã xử lý xong: sum_họp
Đã xử lý xong: nói_lắp
Đã xử lý xong: nói_móc
Đã xử lý xong: vây_ép
Đã xử lý xong: phong_toả
Đã xử lý xong: diễu_hành
Đã xử lý xong: tuần_hành
Đã xử lý xong: cao_trào
Đã xử lý xong: thoái_trào
Đã xử lý xong: thịnh
Đã xử lý xong: suy
Đã xử lý xong: con_voi
Đã xử lý xong: ngà
Đã xử lý xong: cá_chuối
Đã xử lý xong: cá_quả
Đã xử lý xong: đổ_bê_tông
Đã xử lý xong: biếu_xén
Đã xử lý xong: con_đẻ
Đã xử lý xong: con_ruột
Đã xử lý xong: uỷ_nhiệm
Đã xử lý xong: phân_c

### 3.Lưu ra file JSON

In [8]:
with open("../data/BabelNet/babelnet_results3.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)